In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

In [2]:
df_train = pd.read_csv('/root/tempfile/train_final191800.csv')
df_test = pd.read_csv('/root/tempfile/test_final191800.csv')

In [4]:
for i in df_train.columns:
    if df_train[i].dtype == 'object':
        print(i)

card_id
unauth_isholiday_mean
new_isholiday_mean


In [10]:
df_train['auth_isholiday_mean']

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
201887    False
201888    False
201889    False
201890    False
201891    False
201892    False
201893    False
201894    False
201895    False
201896    False
201897    False
201898    False
201899    False
201900    False
201901    False
201902    False
201903    False
201904    False
201905    False
201906    False
201907    False
201908    False
201909    False
201910    False
201911    False
201912    False
201913    False
201914    False
201915    False
201916    False
Name: auth_isholiday_mea

In [ ]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]

In [ ]:
def regression_cv(param, data, cv):
    score = lgb.cv( 
        param, 
        data, 
        nfold=cv,
        stratified=False, 
        shuffle=True,
        metrics='rmse',
        early_stopping_rounds=50,
        verbose_eval=False, 
        show_stdv=False)
    return score['rmse-mean'][-1]


def hyperopt_train_test(params):
    global df_train
    
    param = {
        'objective':'regression',
        "boosting": "gbdt",
    }
    param['max_depth'] = int(params['max_depth'])
    param['num_leaves'] = int(params['num_leaves'])
    param['min_data_in_leaf'] = int(params['min_data_in_leaf'])
    param['reg_alpha'] = params['reg_alpha']
    param['reg_lambda'] = params['reg_lambda']
    param['feature_fraction'] = params['feature_fraction']
    
    df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
    data_train = lgb.Dataset(df_train[df_train_columns], label=df_train['target'])
    return regression_cv(param, data_train, cv=5)


def f(params):
    global best
    score = hyperopt_train_test(params)
    if -score > best:
        best = -score
        print('new best:', -best, params)
    return {'loss': score, 'status': STATUS_OK}